# PINN #1

## Import Relevant Packages

In [1]:
import pandas as pd
import sciann as sn
import numpy as np
from sklearn.preprocessing import MinMaxScaler

---------------------- SCIANN 0.6.1.1 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 


## Load Data from Data Folder

In [2]:
import glob

df = pd.concat(map(pd.read_csv, glob.glob('/Users/prathiklolla/Documents/Research/Research/data/*.csv')))
df

,Qf,Time,X,Y,Alpha,Temp(degC)
0,520,0.0,0.5,2.00,2.671200e-07,20.000000
1,520,0.0,1.0,2.00,2.671200e-07,20.000000
2,520,0.0,1.5,2.00,2.671200e-07,20.000000
3,520,0.0,2.0,2.00,2.671200e-07,20.000000
4,520,0.0,2.5,2.00,2.671200e-07,20.000000
...,...,...,...,...,...,...
3095,600,15.0,50.0,1.92,9.769047e-01,160.016833
3096,600,15.0,50.0,1.94,9.769034e-01,160.013898
3097,600,15.0,50.0,1.96,9.769021e-01,160.011001
3098,600,15.0,50.0,1.98,9.769008e-01,160.008143


In [3]:
# Get only data points where Qf = 500
df = df.loc[df['Qf'] == 500]

In [4]:
# Shuffle the data so it's easy for the model to learn
df = df.sample(frac=1).reset_index(drop=True)
df

,Qf,Time,X,Y,Alpha,Temp(degC)
0,500,4.0,0.0,0.92,0.192741,58.012910
1,500,1.0,31.5,2.00,0.015662,30.252878
2,500,12.5,50.0,0.86,0.906700,121.750354
3,500,9.5,50.0,1.50,0.749595,101.401279
4,500,2.0,50.0,1.72,0.049735,39.894773
...,...,...,...,...,...,...
15495,500,12.5,40.0,1.00,0.909127,122.498165
15496,500,11.0,0.5,0.00,0.847833,112.180200
15497,500,4.0,49.5,1.00,0.192737,58.015221
15498,500,7.0,50.0,0.86,0.511167,82.725005


## Split Data into Train and Test

In [5]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)

In [6]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df[df.isna().any(axis=1)]

,Qf,Time,X,Y,Alpha,Temp(degC)


In [7]:
train_df.head()

,Qf,Time,X,Y,Alpha,Temp(degC)
13767,500,13.0,50.0,0.98,0.921036,124.868565
7897,500,2.5,50.0,1.58,0.080623,44.488788
10326,500,6.0,24.5,1.00,0.406029,75.303841
1611,500,5.0,9.0,1.00,0.291102,66.898555
3499,500,15.0,50.0,1.40,0.956078,136.729062


## Initialize MinMaxScalers

In [8]:
t_scaler = MinMaxScaler(feature_range = (1e-6,1))
x_scaler = MinMaxScaler(feature_range = (1e-6,1))
y_scaler = MinMaxScaler(feature_range = (1e-6,1))

## Scale inputs

In [9]:
t_np = train_df['Time'].to_numpy().reshape(-1,1)
x_np = train_df['X'].to_numpy().reshape(-1,1)
y_np = train_df['Y'].to_numpy().reshape(-1,1)

In [10]:
t_np = t_scaler.fit_transform(train_df['Time'].to_numpy().reshape(-1,1))
x_np = x_scaler.fit_transform(train_df['X'].to_numpy().reshape(-1,1))
y_np = y_scaler.fit_transform(train_df['Y'].to_numpy().reshape(-1,1))

## Define Targets

In [11]:
T_np = train_df['Temp(degC)'].to_numpy().reshape(-1,1)
a_np = train_df['Alpha'].to_numpy().reshape(-1,1)

## Define Inputs and Targets for PINN

In [12]:
# Define the necessary inputs and parameter using Variable and Parameter
t = sn.Variable('t')
x = sn.Variable('x')
y = sn.Variable('y')

# Define Neural Network Approximations
T = sn.Functional('T', [t,x,y], 6*[20], 'tanh')
a = sn.Functional('a', [t,x,y], 6*[20], 'tanh')

## Define Constants and PDEs

In [13]:
# Constants
TOL = 0.000001

H = 352.1 #1
A = 2e5 #2
E_a = 51100 #3 
n = 1.927 #4
p = 980 #5
c_p = 1440 #6
k = 1.8 #7
R = 8.314 #8
T_o = 20.0 #9
a_o = 0 + TOL #10
h = 1.8 #11
Q = 500 #12

# PDEs
from sciann.utils.math import diff, sign, exp, pow

L1 = (k/(p*c_p))*diff(T, x, order=2) + (k/(p*c_p))*diff(T, y, order=2) + (H/c_p)*diff(a, t) - diff(T, t)
L2 = A*exp(-E_a/(R*T))*pow(1-a, n)*(1 + 0.36*a) - diff(a, t)

## Define Initial & Boundary Conditions

In [14]:
# t: [0,15]
# x: [0,50]
# y: [0,2]
# Q: [500,700]

tmin = 0 + TOL
Xmin = 0 + TOL
Xmax = 1
Ymin = 0 + TOL
Ymax = 1

# Boundary Conditions
L3 = (k*diff(T, x))*(1-sign(x - Xmin)) + (h*T)*(1-sign(x - Xmin)) - (h*T_o)*(1-sign(x - Xmin))

L4 = (k*diff(T, x))*(1+sign(x - Xmax)) + (h*T)*(1+sign(x - Xmax)) - (h*T_o)*(1+sign(x - Xmax))

L5 = (k*diff(T, y))*(1-sign(y - Ymin)) + (h*T)*(1-sign(y - Ymin)) - (h*T_o)*(1-sign(y - Ymin))

L6 = (k*diff(T,y) - Q)*(1+sign(y - Ymax))

# Initial Conditions
L7 = (1-sign(t - tmin))*(T_o)

L8 = (1-sign(t - tmin))*(a_o)

# Other Constraints
L9 = sn.Data(T)

L10 = sn.Data(a)

## Build the Model

In [15]:
m = sn.SciModel([t,x,y], 
                [L1,L2,L3,L4,L5,L6,L7,L8,L9,L10],
                optimizer="adam")

In [16]:
h = m.train(
    [t_np, x_np, y_np], 
    ['zeros','zeros','zeros','zeros','zeros','zeros','zeros','zeros', T_np, a_np], 
    epochs=8000,
    batch_size=64,
    learning_rate=0.001,
    verbose = 0,
    shuffle=False)


Total samples: 12400 
Batch size: 64 
Total batches: 194 

Batch 0: Invalid loss, terminating training
